---
title : "Overstromingsrisico scenarios"
execute:
    output: asis
---

Scenarios worden gebruikt om de mogelijke gevolgen van overstromingen in kaart te brengen. <br>
Hiermee kan de kans uit een fragilitycruve gecombineerd worden met de gevolgen die bepaald zijn bij een scenario om zo een beter inzicht te krijgen in het totale risico. <br>
Dit kan helpen om prioriteiten te stellen op basis van een totaal beeld en niet alleen op basis van kans of gevolg afzonderlijk. <br>
Om de scenarios te kunnen combineren moeten er een aantal stappen worden doorlopen:
- Omrekenen van dijkvakkansen naar trajectdeelkansen ([CalculateFloodScenarioProbability](#sec-CalculateFloodScenarioProbability))
- Belasting per deeltraject bepalen ([LoadFromFloodScenarioProbabilit](#sec-LoadFromFloodScenarioProbabilit))
- Met de belasting een bijpassende overstromingscenario selecteren ([SelectFloodScenarioFromLoad ](#sec-SelectFloodScenarioFromLoad))
- Van het scenario en de curve een geagregeerd risico bepalen ([CalculateFloodRisk](#sec-CalculateFloodRisk))


In [ ]:
from pathlib import Path

from toolbox_continu_inzicht.base.config import Config
from toolbox_continu_inzicht.base.data_adapter import DataAdapter

In [ ]:
path = Path.cwd() / "data_sets" / "7.flood_scenarios"
config = Config(config_path=path / "test_calculate_flood_scenario_probability.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
from toolbox_continu_inzicht.flood_scenarios import CalculateFloodScenarioProbability

## CalculateFloodScenarioProbability{sec-CalculateFloodScenarioProbability}

Dit voorbeeld laat zien hoe met behulp van de `CalculateFloodScenarioProbability`  dijkvakkansen naar trajectdeelkansen kunnen worden omgezet. <br>
Hiervoor is de gecombineerde faalkans per dijkvak nodig en een mapping van dijkvakken naar dijktraject delen van daar naar de overstromingsscenario gebieden.
```yaml|
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [-24,0,24,48]

DataAdapter:
    default_options:
        csv:
            sep: ","
    combined_failure_probability_data:
        type: csv
        file: "combined_failure_probability_data.csv"
    section_id_to_segment_id:
        type: csv
        file: "section_id_to_segment_id.csv"
    segment_failure_probability:
        type: csv
        file: "segment_failure_probability.csv"
```

In [ ]:
calculate_flood_scenario_probability = CalculateFloodScenarioProbability(
    data_adapter=data_adapter
)
calculate_flood_scenario_probability.run(
    input=["combined_failure_probability_data", "section_id_to_segment_id"],
    output="segment_failure_probability",
)
# TODO: segment to dikesystem, only needed later on for risk = probability * damage

In [ ]:
calculate_flood_scenario_probability.df_in_sections_to_segment

In [ ]:
calculate_flood_scenario_probability.df_in_grouped_sections_failure_probability

In [ ]:
calculate_flood_scenario_probability.df_out

In [ ]:
from toolbox_continu_inzicht.flood_scenarios import LoadFromFloodScenarioProbability

## LoadFromFloodScenarioProbability{sec-LoadFromFloodScenarioProbabilit}

Nu we de faalkansen per trajectdeel hebben kunnen we met de `LoadFromFloodScenarioProbability` de belasting per trajectdeel bepalen. <br>
Dit doen we om een bijpassende overstromingsscenario te kunnen selecteren. <br>
Met de input van de vorige stap `segment_failure_probability`, maken we een koppeling naar de bijhorende bres en fragilitycurve die representatief is voor een deeltraject met de tabel `breach_id_to_segment_id`. <br>
Deze `representative_section_id_fragilitycurve` kan vervolgens uit de data van de curves in `section_failure_probability_data` worden gehaald. <br>
Met de curve wordt de hydraulische belasting bepaald voor het deeltraject.

```yaml
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [ -24, 0, 24, 48 ]

DataAdapter:
    default_options:
        csv:
            sep: ","
    segment_failure_probability:
        type: csv
        file: "segment_failure_probability.csv"
    breach_id_to_segment_id:
        type: csv
        file: "breach_id_to_segment_id.csv"
    section_failure_probability_data:
        type: csv
        file: "fragility_curve_multi_section.csv"
    flood_scenario_load_resultaten:
        type: csv
        path: "hidden_flood_scenario_load_resultaten.csv"

```

In [ ]:
config = Config(config_path=path / "test_load_from_flood_scenario_probability.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
data_adapter.input("segment_failure_probability")

In [ ]:
data_adapter.input("breach_id_to_segment_id")

In [ ]:
data_adapter.input("section_failure_probability_data")

In [ ]:
load_from_flood_scenario_probability = LoadFromFloodScenarioProbability(
    data_adapter=data_adapter
)
load_from_flood_scenario_probability.run(
    input=[
        "segment_failure_probability",
        "breach_id_to_segment_id",
        "section_failure_probability_data",
    ],
    output="flood_scenario_load_resultaten",
)

In [ ]:
data_adapter.input("flood_scenario_load_resultaten")

In [ ]:
df = data_adapter.input("section_failure_probability_data")

In [ ]:
import numpy as np
from toolbox_continu_inzicht.utils.interpolate import (
    log_y_interpolate_1d,
    log_x_interpolate_1d,
)


df_fc = df[df["section_id"] == 1][["hydraulicload", "failure_probability"]]
x, fp = df_fc["hydraulicload"].to_numpy(), df_fc["failure_probability"].to_numpy()
y = np.array(sorted(list(fp) + [0.183]))
new_x = log_y_interpolate_1d(y, x, fp, ll=1e-200)

In [ ]:
new_x

In [ ]:
x

In [ ]:
y

In [ ]:
import matplotlib.pyplot as plt

plt.plot(x, fp, label="data points")
plt.plot(new_x, y, label="interpolated")
plt.yscale("log")

In [ ]:
df[df["section_id"] == 1][["hydraulicload", "failure_probability"]].set_index(
    "hydraulicload"
).plot(logy=True)

In [ ]:
# todo: add to fragility curve tests

In [ ]:
from toolbox_continu_inzicht.utils.interpolate import (
    log_y_interpolate_1d,
    beta_y_interpolate_1d,
)

In [ ]:
import numpy as np

x_new = np.array([2, 3, 4])
y = np.array([1 / 1000, 1 / 10])
x = np.array([2, 4])
y_new = np.array([1 / 1000, 1 / 100, 1 / 10])

In [ ]:
log_x_interpolate_1d(x_new, x, y)

In [ ]:
log_y_interpolate_1d(y_new, x, y)

In [ ]:
beta_y_interpolate_1d(y_new, x, y)